# MEME CLASSIFIER - TASK 1

Useful links:
- [TRANSFORMERS DOCS](https://huggingface.co/docs/transformers)
- [TASK PAGE](https://propaganda.math.unipd.it/semeval2021task6/)
- [TASK GITHUB](https://github.com/di-dimitrov/SEMEVAL-2021-task6-corpus#task-description)
- [MULTI-LABEL CLASSIFICATION ARTICLE](https://medium.com/huggingface/multi-label-text-classification-using-bert-the-mighty-transformer-69714fa3fb3d)
- [ADDING CUSTOM LAYERS TO BERT ARTICLE](https://towardsdatascience.com/adding-custom-layers-on-top-of-a-hugging-face-model-f1ccdfc257bd)
- [BERT FINE-TUNING KAGGLE](https://www.kaggle.com/code/jaskaransingh/bert-fine-tuning-with-pytorch/notebook)
- [HOW DO BERT WORKS](https://www.exxactcorp.com/blog/Deep-Learning/how-do-bert-transformers-work)
- [TRANSFORMERS HARVARD GUIDE](http://nlp.seas.harvard.edu/annotated-transformer/)
- [INTRODUCTION ON THRESHOLD MOVING](https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/)
- [DEALING WITH UNBALANCED DATASETS](https://medium.com/dataseries/how-to-deal-with-unbalanced-dataset-in-binary-classification-part-3-e580d8d09883)
- [FOCAL LOSS PAPER](https://openaccess.thecvf.com/content_ICCV_2017/papers/Lin_Focal_Loss_for_ICCV_2017_paper.pdf)

### PACKAGES INSTALLATION

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 25.4 MB/s 
     |████████████████████████████████| 182 kB 71.4 MB/s 
     |████████████████████████████████| 7.6 MB 62.1 MB/s 


### DRIVE LINKING

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### CONSTANTS

In [3]:
LABELS = {
    'Black-and-white Fallacy/Dictatorship': 0,
    'Loaded Language': 1, 
    'Name calling/Labeling': 2, 
    'Slogans': 3, 
    'Smears': 4, 
    'Causal Oversimplification': 5, 
    'Appeal to fear/prejudice': 6, 
    'Exaggeration/Minimisation': 7, 
    'Reductio ad hitlerum': 8, 
    'Repetition': 9, 
    'Glittering generalities (Virtue)': 10, 
    "Misrepresentation of Someone's Position (Straw Man)": 11, 
    'Doubt': 12, 
    'Obfuscation, Intentional vagueness, Confusion': 13, 
    'Whataboutism': 14, 
    'Flag-waving': 15, 
    'Thought-terminating cliché': 16, 
    'Presenting Irrelevant Data (Red Herring)': 17, 
    'Appeal to authority': 18, 
    'Bandwagon': 19,
}
N_LABELS = len(LABELS)

### IMPORTS

In [4]:
import os
from datetime import datetime

import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tabulate import tabulate
import seaborn as sn

from tqdm.notebook import tqdm
from IPython.display import clear_output

import torch 
from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import AutoTokenizer
from transformers import AutoModel, AutoConfig
from transformers import PreTrainedModel, BertModel, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers.utils import logging

from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_curve

### DATASET

####  DATASET LOADING

In [5]:
def load_datasets():

    def load_json2pandas(path): 
        
        with open(path) as f:
            data = json.load(f)
        return pd.DataFrame(data)

    folder = "drive/MyDrive/DeepLearning/Dataset/task1/"

    train = load_json2pandas(folder + "training_set_task1.txt")
    test = load_json2pandas(folder + "test_set_task1.txt")
    dev = load_json2pandas(folder + "dev_set_task1.txt")

    return train, test, dev

#### LABELS ENCODING

In [6]:
def one_hot_encoder(df):

    def encode(labels):

        ohe_label = [0] * N_LABELS
        for l in labels:
            ohe_label[LABELS[l]] = 1

        return ohe_label

    return pd.DataFrame([encode(labels) for labels in df["labels"]])

#### EXAMPLES ON THE DATASET

In [7]:
train, _, _ = load_datasets()
train.head()

,id,labels,text
0,128,[Black-and-white Fallacy/Dictatorship],THERE ARE ONLY TWO GENDERS\n\nFEMALE \n\nMALE\n
1,189,[],This is not an accident!
2,96,"[Loaded Language, Name calling/Labeling, Sloga...",SO BERNIE BROS HAVEN'T COMMITTED VIOLENCE EH?\...
3,154,"[Causal Oversimplification, Loaded Language, N...",PATHETIC\n\nThe Cowardly Asshole\nWeak Failure...
4,15,[],WHO TRUMP REPRESENTS\n\nWHO DEMOCRATS REPRESENT\n


In [8]:
train_ohe_labels = one_hot_encoder(train)
train_ohe_labels.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
train = pd.concat([train, train_ohe_labels], axis=1)
train.head()

,id,labels,text,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,18,19
0,128,[Black-and-white Fallacy/Dictatorship],THERE ARE ONLY TWO GENDERS\n\nFEMALE \n\nMALE\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,189,[],This is not an accident!,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,96,"[Loaded Language, Name calling/Labeling, Sloga...",SO BERNIE BROS HAVEN'T COMMITTED VIOLENCE EH?\...,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,154,"[Causal Oversimplification, Loaded Language, N...",PATHETIC\n\nThe Cowardly Asshole\nWeak Failure...,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,15,[],WHO TRUMP REPRESENTS\n\nWHO DEMOCRATS REPRESENT\n,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### DATASET

In [10]:
class MemeDataset():


    def __init__(self, texts, labels, tokenizer):

        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
    
    def __len__(self):

        return len(self.texts)

    def __getitem__(self, index):
        
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(
            text,
            None,
            add_special_tokens=True,
            padding="max_length",
            truncation=True)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long),
        }

#### WRAPPERS

In [11]:
def process_datasets(train, test, dev):

    def process_dataset(df):
        
        ohe_labels = one_hot_encoder(df)
        return pd.concat([df, ohe_labels], axis=1)

    train_ps = process_dataset(train)
    test_ps = process_dataset(test)
    dev_ps = process_dataset(dev)

    return train_ps, test_ps, dev_ps

def build_datasets(train, test, dev):

    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    train_ds = MemeDataset(
        train.text.tolist(), train[range(N_LABELS)].values.tolist(), tokenizer)
    test_ds = MemeDataset(
        test.text.tolist(), test[range(N_LABELS)].values.tolist(), tokenizer)
    dev_ds = MemeDataset(
        dev.text.tolist(), dev[range(N_LABELS)].values.tolist(), tokenizer)
    
    return train_ds, test_ds, dev_ds

def build_dataloaders(train_ds, test_ds, dev_ds):
    
    train_dl = DataLoader(train_ds, batch_size=14, shuffle=True, num_workers=2)
    test_dl = DataLoader(test_ds, batch_size=14, shuffle=True, num_workers=2)
    dev_dl = DataLoader(dev_ds, batch_size=14, shuffle=True, num_workers=2)

    return train_dl, test_dl, dev_dl 

### MODEL

In [12]:
class MemeClassifier(nn.Module):
    """Our classifier for Task 1.
    
    It predicts a label for every class based on the output
    of a Bert transformer, then fed to a linear layer."""

    def __init__(self, n_classes, do_prob=0.2, bert_path="bert-base-uncased"):

        super(MemeClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_path)
        self.step_scheduler_after = "batch"
        self.dropout = nn.Dropout(do_prob)
        self.classifier = nn.Linear(768, n_classes)
        self.sigmoid = torch.sigmoid

    def forward(self, input_ids, attention_mask=None):

        output = self.bert(input_ids, attention_mask=attention_mask)["pooler_output"]
        output = self.dropout(output)
        logits = self.classifier(output)
        output = self.sigmoid(logits)
        return output

In [13]:
def save_model(model, thresholds, path):
    """Saves the models with their thresholds."""

    torch.save({
            'model_state_dict': model.state_dict(),
            'thresholds': thresholds,
            }, path)

def load_model(path, device):
    """Loads the models saved with their thresholds."""

    model = MemeClassifier(N_LABELS)
    model = nn.DataParallel(model)

    checkpoint = torch.load(path, map_location=device)

    model.load_state_dict(checkpoint['model_state_dict'])
    thresholds = checkpoint['thresholds']
    
    return model, thresholds

### OPTIMIZER

In [14]:
def build_optimizer(model, lr=3e-5):

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(
                    nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(
                    nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    opt = AdamW(optimizer_parameters, lr=lr)
    return opt

### PLOTTER

In [15]:
class Plotter():
    """Keeps track of the metrics and create plots."""

    def __init__(self):

        self.data = {}

    def add(self, d):

        for metric in d:
            if metric in self.data: 
                self.data[metric].append(d[metric])
            else: 
                self.data[metric] = [d[metric]]

    def plot_metric(self, metric):

        y = self.data[metric]
        x = range(1, len(y) + 1)

        plt.figure()
        plt.title(metric)
        plt.xlabel('epoch')
        plt.plot(x, y) 

    def plot(self):

        for metric in self.data:
            self.plot_metric(metric)
            plt.show()

    def save(self, dir_name):

        if not os.path.isdir(dir_name): return
            
        for metric in self.data:
            self.plot_metric(metric)

            plt.savefig("".join([dir_name, metric]))
            plt.close()

### CONSOLE WINDOW

In [16]:
class ConsoleWindow():
    """Controls the console output and formats it as a window"""

    def __init__(self):

        self.border_up = 1
        self.border_down = 1
        self.border_left = 1
        self.border_right = 1

        self.frame_up = 1
        self.frame_down = 1
        self.frame_left = 1
        self.frame_right = 1

        self.border_char = " "
        self.frame_char = "#"

        self.__align_left = 0
        self.__align_right = 1
        self.__align_center = 2
        self.__align = self.__align_left

        self.__showing = False

        self.__reset_memory()

    def __reset_memory(self):

        self.__lines = []
        self.__addresses = {}

    def __clear_window(self, wait=True): 
        
        if not self.__showing: return

        clear_output(wait=wait)
        self.__showing = False

    def __max_line_len(self):

        if not self.__lines: 
            return 0
        return max([len(line) for line in self.__lines])

    def __print_window(self): 

        if self.__showing: return

        max_len_line = self.__max_line_len()
        window_width = (max_len_line
                        + self.border_left
                        + self.border_right
                        + self.frame_left
                        + self.frame_right)
        
        def print_line(line): 

            print(self.frame_char * self.frame_left, end="") 
            print(self.border_char * self.border_left, end="")

            l, r = 0, 0
            if self.__align == self.__align_left:
                l, r = 0, max_len_line - len(line)
            elif self.__align == self.__align_right:
                l, r = max_len_line - len(line), 0
            else:    # center
                l = (max_len_line - len(line)) // 2
                r = (max_len_line - len(line)) - l

            print((" "*l) + line + (" "*r), end="") 

            print(self.border_char * self.border_right, end="")
            print(self.frame_char * self.frame_right)

        def print_frame_line(): 

            print(self.frame_char * window_width)

        def print_border_line(): 

            print_line(self.border_char * self.__max_line_len())
        
        for h in range(self.frame_up):
            print_frame_line()
        
        for h in range(self.border_up):
            print_border_line()

        for line in self.__lines:
            print_line(line)

        for h in range(self.border_down):
            print_border_line()

        for h in range(self.frame_down):
            print_frame_line()

        self.__showing = True

    def __refresh_window(self):

        self.__clear_window()
        self.__print_window()

    def replace(self, address, line):

        if address not in self.__addresses:
            return self.print(line, address)

        self.__lines[self.__addresses[address]] = line
        self.__refresh_window()


    def print(self, line, address=None):

        if not isinstance(line, str):
            raise TypeError('"line" has to be of type string.')

        if address in self.__addresses:
            return self.replace(address, line)

        if address is not None:
            self.__addresses[address] = len(self.__lines)

        self.__lines.append(line)

    def clear_all(self):

        self.__clear_window(wait=False)
        self.__reset_memory()

    def show(self): self.__refresh_window()

    def align_on_left(self): self.__align = self.__align_left
    def align_on_right(self): self.__align = self.__align_right
    def align_on_center(self): self.__align = self.__align_center


class ProgressBar():

    
    def __init__(self, num_iter, bar_len=20):

        self.__tot = num_iter
        self.__progress = 0

        self.__bar_len = bar_len

        self.full_char = "="
        self.empty_char = " "

    def __repr__(self) -> str:

        full_chars = int(self.__bar_len * (self.__progress / self.__tot))
        void_chars = self.__bar_len - full_chars

        return "[{}{}] [{}/{}]".format(
            self.full_char * full_chars, 
            self.empty_char * void_chars,
            str(self.__progress),
            str(self.__tot))

    def __str__(self) -> str: 
        
        return self.__repr__()

    def step(self, n=1): self.__progress += n

### TRAINER

In [17]:
class FocalLoss(nn.modules.loss._WeightedLoss):


    def __init__(self, weight=None, gamma=2):

        super(FocalLoss, self).__init__(weight)
        self.gamma = gamma
        self.weight = weight 

    def forward(self, input, target):

        ce_loss =  F.binary_cross_entropy(
            input, target, reduction='none', weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

In [18]:
def find_opt_threshold(outputs, labels):

    def to_labels(outputs, threshold): 
        return (outputs >= threshold).astype('int')

    thresholds = np.arange(0.1, 1, 0.001)

    scores = [
        f1_score(
            labels, 
            to_labels(outputs, t), 
            average=None, 
            zero_division=1
        ) for t in thresholds
    ]

    idxs = np.argmax(scores, axis=0)

    return np.take(thresholds, idxs)

def find_opt_threshold_gmean(outputs, labels):

    def get_t(lab, out):

        fpr, tpr, thresholds = roc_curve(lab, out)
        gmeans = np.sqrt(tpr * (1-fpr))
        return thresholds[np.argmax(gmeans)]

    return np.array(
        [get_t(lab, out) for lab, out in zip(labels.T, outputs.T)])

def apply_threshold_per_class(outputs, thresholds):

    y = []

    for output in outputs: 
        y.append([0 if p < t else 1 for p, t in zip(output, thresholds)])

    return np.array(y)

def apply_threshold_to_all(outputs, threshold):

    return (outputs >= threshold).astype('int')

def get_metrics(preds, labels):    

    f1_micro = f1_score(labels, preds, average='micro', zero_division=1)
    f1_macro = f1_score(labels, preds, average='macro', zero_division=1)

    return {'f1-micro': f1_micro, 'f1-macro': f1_macro}

In [19]:
def eval(dataloader, model, device, loss_fn=None, thresholds=None):

    eval_loss = 0.0
    model.eval()

    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in enumerate(dataloader):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids, mask)

            if loss_fn is not None:
                loss = loss_fn(outputs, targets)
                eval_loss += loss.item()

            fin_targets.extend(targets)
            fin_outputs.extend(outputs)

    fin_outputs = torch.stack(fin_outputs)
    fin_targets = torch.stack(fin_targets)
    fin_outputs = fin_outputs.cpu().detach().numpy()
    fin_targets = fin_targets.cpu().detach().numpy()

    if thresholds is None:
        thresholds = find_opt_threshold(fin_outputs, fin_targets)
    
    pred = apply_threshold_per_class(fin_outputs, thresholds)
    pred_non_opt = apply_threshold_to_all(fin_outputs, 0.5)
        
    return {
        "loss": eval_loss,
        "output": fin_outputs,
        "pred": pred,
        "pred_non_opt": pred_non_opt,
        "true": fin_targets,
        "thresholds": thresholds,
    }


class Trainer():
    """Trains a model while printing results."""

    def __init__(self, model, optimizer, scheduler, device, loss_fn):

        self.net = model
        self.opt = optimizer 
        self.scheduler = scheduler
        self.device = device
        self.loss_fn = loss_fn

        self.window = ConsoleWindow()
        self.plotter = Plotter()

        self.window.frame_char = '#'
        self.window.border_char = '░'
        self.window.border_left = 2
        self.window.border_right = 2
        self.window.align_on_center()

    def eval(self, dataloader, thresholds):

        self.window.print("Evaluating...", "eval")
        self.window.show()
        res = eval(
            dataloader, 
            self.net, 
            self.device, 
            loss_fn=self.loss_fn, 
            thresholds=thresholds)
        self.window.print("", "eval")

        return res

    def train(self, train_dl, val_dl, epochs, save_dir=None, dir_name=None):

        def train_fn():

            train_loss = 0.0
            self.net.train()

            fin_outputs = []
            fin_targets = []

            bar = ProgressBar(len(train_dl))
            self.window.print(str(bar), "tbar")
            self.window.show()
            for bi, d in enumerate(train_dl):
                ids = d["ids"]
                mask = d["mask"]
                targets = d["labels"]

                ids = ids.to(self.device, dtype=torch.long)
                mask = mask.to(self.device, dtype=torch.long)
                targets = targets.to(self.device, dtype=torch.float)
                outputs = self.net(ids, mask)
                self.opt.zero_grad()

                loss = loss_fn(outputs, targets)
                loss.backward()
                train_loss += loss.item()
                self.opt.step()
                self.scheduler.step()

                fin_targets.extend(targets)
                fin_outputs.extend(outputs)

                bar.step()
                self.window.print(str(bar), "tbar")
                self.window.show()

            fin_outputs = torch.stack(fin_outputs)
            fin_targets = torch.stack(fin_targets)
            fin_outputs = fin_outputs.cpu().detach().numpy()
            fin_targets = fin_targets.cpu().detach().numpy() 

            thresholds = find_opt_threshold(fin_outputs, fin_targets)
                
            return train_loss, thresholds
        
        if save_dir is not None and os.path.isdir(save_dir):
            datestring = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if dir_name is None:  
                save_dir = "".join([save_dir, datestring, "/"])
            elif os.path.isdir("".join([save_dir, dir_name, "/"])):
                save_dir = "".join([save_dir, dir_name, datestring, "/"])
                self.window.print(
                    "dir_name specicified already exists. Saving as {}".format(
                        save_dir))
            else:
                save_dir = "".join([save_dir, dir_name, "/"])

            os.mkdir(save_dir)

        best_val_loss = 100
        best_val_micro_non_opt = 0
        best_val_micro = 0

        for epoch in tqdm(range(1, epochs+1)):
            self.window.print("Epoch {}/{}".format(epoch, epochs), "E")
            self.window.show()

            train_loss, thresholds = train_fn()

            res = self.eval(val_dl, thresholds)

            eval_loss = res['loss']
            pred = res['pred']
            pred_non_opt = res['pred_non_opt']
            true = res['true']

            avg_train_loss = train_loss / len(train_dl)
            avg_val_loss = eval_loss / len(val_dl)

            metrics = get_metrics(pred, true)
            metrics_non_opt = get_metrics(pred_non_opt, true)

            metrics['train_loss'] = avg_train_loss
            metrics['val_loss'] = avg_val_loss
            metrics['f1-micro_non_opt'] = metrics_non_opt['f1-micro']
            metrics['f1-macro_non_opt'] = metrics_non_opt['f1-macro']

            self.plotter.add(metrics)

            self.window.print(
                "Average Train loss: {}".format(avg_train_loss), "TL")
            self.window.print(
                "Average Valid loss: {}".format(avg_val_loss), "VL")
            self.window.print(
                "F1-micro: {}".format(metrics['f1-micro']), "MI")
            self.window.print(
                "F1-macro: {}".format(metrics['f1-macro']), "MA")

            if save_dir is not None:
                self.window.print("", "sp")
                save_msg = "Model saved with best {} is: {:.4f}"
                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    save_model(
                        self.net, thresholds, save_dir + "best_model_loss.pt")
                    self.window.print(
                        save_msg.format("val_loss", best_val_loss), "SL")

                if metrics['f1-micro_non_opt'] > best_val_micro_non_opt:
                    best_val_micro_non_opt = metrics['f1-micro_non_opt']
                    save_model(
                        self.net, 
                        thresholds, 
                        save_dir + "best_model_micro_non_opt.pt")
                    self.window.print(
                        save_msg.format(
                            "non_opt_micro", best_val_micro_non_opt), "SMINO")

                if metrics['f1-micro'] > best_val_micro:
                    best_val_micro = metrics['f1-micro']
                    save_model(
                        self.net, thresholds, save_dir + "best_model_micro.pt")
                    self.window.print(
                        save_msg.format("micro", best_val_micro), "SMI")

                self.plotter.save(save_dir) 

            self.window.show()

    def plot_metrics(self):

        self.plotter.plot()

### TRAINING

In [20]:
def get_alpha(data):

    labels = torch.tensor(one_hot_encoder(data).values.tolist())

    n_per_class = torch.sum(labels, dim=0)

    return max(n_per_class) / n_per_class

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(128)

In [22]:
train, test, dev = load_datasets()

train, test, dev = process_datasets(train, test, dev)

train_ds, test_ds, dev_ds = build_datasets(train, test, dev)
train_dl, test_dl, dev_dl = build_dataloaders(train_ds, test_ds, dev_ds)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
bert_unfrozen = "drive/MyDrive/DeepLearning/Models/\
Pretrained_unfrozen/checkpoint-40000"
bert_frozen = "drive/MyDrive/DeepLearning/Models/\
Pretrained_frozen/checkpoint-40000"

model = MemeClassifier(N_LABELS, bert_path=bert_unfrozen)

# alpha = get_alpha(train).to(device)
loss_fn = nn.BCELoss()
# loss_fn = FocalLoss()

model.to(device)
model = nn.DataParallel(model)

Some weights of the model checkpoint at drive/MyDrive/DeepLearning/Models/Pretrained_unfrozen/checkpoint-40000 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at drive/MyDrive/DeepLearning/Models/Pretrained_unfrozen/

In [24]:
TRAIN = False
save_dir = "drive/MyDrive/DeepLearning/Task1/"

epochs = 50
lr = 4e-5

total_steps = len(train_dl) * epochs

optimizer = build_optimizer(model, lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

trainer = Trainer(model, optimizer, scheduler, device, loss_fn)

if TRAIN:
    trainer.train(
        train_dl, 
        dev_dl, 
        epochs, 
        dir_name="unfrozen-final", 
        save_dir=save_dir)
    
    trainer.plot_metrics()

### TESTER

In [25]:
def add_threshold_uncertainty(thresholds, degree=2):
    """Adds uncertainty to the thresholds by flatting them towards
    0.5, following a polynomial function. The 'degree' parameter chooses
    the degree of the polynomial. If it is set to 1, the 
    thresholds are all set to 0.5 (max uncertainty)."""

    assert (int(degree) == degree and 
            degree > 0), '"degree" has to be a positive integer.'

    def uncertainty(x):
        
        u = - (x - 0.5)**degree
        if degree % 2:
            return u
        else:
            return u if x > 0.5 else -u
    
    return thresholds + np.vectorize(uncertainty)(thresholds)

x = np.arange(0, 1.1, 0.1)
y = add_threshold_uncertainty(x, degree=5)

print("EXAMPLE:")
print("OLD THRESHOLDS:\n", x)
print("\nUNCERTAIN THRESHOLDS:\n", y)

EXAMPLE:
OLD THRESHOLDS:
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

UNCERTAIN THRESHOLDS:
 [0.03125 0.11024 0.20243 0.30032 0.40001 0.5     0.59999 0.69968 0.79757
 0.88976 0.96875]


In [26]:
class Tester():
    """Evaluates a model and prints plots and metrics."""

    def __init__(self, model, dl, device, thresholds=None):

        res = eval(dl, model, device, thresholds=thresholds)

        self.outputs = res['output']
        self.pred = res['pred']
        self.true = res['true']
        self.pred_non_opt = res['pred_non_opt']

        thresholds = add_threshold_uncertainty(thresholds, degree=9)

        res = eval(dl, model, device, thresholds=thresholds)

        self.pred_uncertain = res["pred"]

    def print_f_table(self):

        f = f1_score(
            self.true, self.pred, average=None, zero_division=1)
        
        print(tabulate([f], headers=LABELS.keys(), floatfmt=".4f"))

    def print_fmacro(self):

        print("f_macro: {:10.4f}".format(f1_score(
            self.true, self.pred, average='macro', zero_division=1)))
        print("f_macro w/out threshold moving: {:10.4f}".format(f1_score(
            self.true, self.pred_non_opt, average='macro', zero_division=1)))
        print("f_macro w/ threshold uncertainty: {:10.4f}".format(f1_score(
            self.true, self.pred_uncertain, average='macro', zero_division=1)))

    def print_fmicro(self): 

        print("f_micro: {:10.4f}".format(f1_score(
            self.true, self.pred, average='micro', zero_division=1)))
        print("f_micro w/out threshold moving: {:10.4f}".format(f1_score(
            self.true, self.pred_non_opt, average='micro', zero_division=1)))
        print("f_micro w/ threshold uncertainty: {:10.4f}".format(f1_score(
            self.true, self.pred_uncertain, average='micro', zero_division=1)))
        
    def plot_confusion_matrices(self):

        cms = multilabel_confusion_matrix(self.true, self.pred)

        r = N_LABELS // 4
        c = N_LABELS // r
        fig, axs = plt.subplots(r, c, figsize=(12, 14))
        axs = axs.flat

        titles = [
            l if len(l) < 22 else l[:20] + ".." for l in list(LABELS.keys())]

        for idx, cm in enumerate(cms):
            df_cm = pd.DataFrame(
                cm, index = ["F", "T"], columns = ["F", "T"])
            axs[idx] = sn.heatmap(
                df_cm, annot=True, ax=axs[idx], cmap=plt.cm.Greens)
            axs[idx].title.set_text(titles[idx])
            
        fig.tight_layout()

        fig.show()
        plt.show()

    def show_class_results(self, class_idx, layout='horizontal'):

        if layout != 'vertical' and layout != 'horizontal':
            print("Layout not recognized. Using horizontal.\n\
            Possible layouts: 'horizontal', 'vertical'")

        pred = [p[class_idx] for p in self.pred]
        outs = [p[class_idx] for p in self.outputs]
        true = [p[class_idx] for p in self.true]

        def colored_list(i):
            """Returns a list with true pred and out 
            at idx i colored in:
            red if true[i] == 1 and pred[i] == 0
            yellow if true[i] == 0 and pred[i] == 1
            blue otherwise"""
            c = '\033[94m'
            if true[i] - pred[i] == 1   : c = '\033[91m'
            elif true[i] - pred[i] == -1: c = '\033[93m'

            def color(n, c): return '{}{:.4f}'.format(c, n)

            return [color(true[i], c), color(pred[i], c), color(outs[i], c)]

        headers = ['y_true', 'y_pred', 'output']
        table = [ colored_list(i) for i in range(len(true)) ]

        if layout == 'vertical': 
            print(tabulate(table, headers=headers))
        else: 
            table = list(map(list, zip(*table)))
            print('\033[97moutput | ', end='')
            for o in table[2] : print(o, end='  ')
            print('\033[97m\ny_true | ', end='')
            for o in table[0] : print(o, end='  ')
            print('\033[97m\ny_pred | ', end='')
            for o in table[1] : print(o, end='  ')
            print()

    def show_all_class_results(self):

        for i in range(0, N_LABELS): 
            print("\033[97mClass {} ({}) :".format(list(LABELS.keys())[i], i))
            self.show_class_results(i)
            print()

In [27]:
def test_model(path, test_level=0, model_name=["MODEL"]):

    model, thresholds = load_model(path, device)
    model.to(device)

    res = eval(train_dl, model, device)
    thresholds = res['thresholds']

    tester = Tester(model, test_dl, device, thresholds)

    print("####### TESTING {} #######".format(model_name.upper()))

    tester.print_fmicro()
    print()
    tester.print_fmacro()

    if test_level >= 1:
        tester.print_f_table()
        tester.plot_confusion_matrices()

    if test_level >= 2:
        tester.show_all_class_results()

    print("---------------------------")

def test_models(models_dir, models_names, to_test=None, test_level=0):

    err_msg = '"to_test" has to be a list of string or None'
    if to_test is not None and (not isinstance(to_test, list) or 
                                not all([isinstance(t, str) for t in to_test])):
        raise TypeError(err_msg)

    logging.set_verbosity(40)

    for model_name in models_names:
        dir = "".join([models_dir, model_name])

        f_names = []
        if to_test is None:
            for f in os.listdir(dir):
                if f.endswith(".pt"):
                    f_names.append(f)
        else:
            for m in to_test:
                f_names.append("best_model_{}.pt".format(m))

        for name in f_names:   
            path = "{}/{}".format(dir, name)
            m_name = " ".join([model_name, name[:-3]])

            test_model(path, model_name=m_name, test_level=test_level)

    logging.set_verbosity(20)

### TESTING

In [28]:
models_dir = "drive/MyDrive/DeepLearning/Task1/"
models_names = ['base', 'unfrozen', 'frozen', 'frozen-focal']
to_test = ['micro', 'micro_non_opt']

test_models(models_dir, 
            models_names,
            to_test=to_test,
            test_level=0)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

####### TESTING BASE BEST_MODEL_MICRO #######
f_micro:     0.5431
f_micro w/out threshold moving:     0.5346
f_micro w/ threshold uncertainty:     0.3136

f_macro:     0.1732
f_macro w/out threshold moving:     0.1420
f_macro w/ threshold uncertainty:     0.0711
---------------------------
####### TESTING BASE BEST_MODEL_MICRO_NON_OPT #######
f_micro:     0.5364
f_micro w/out threshold moving:     0.5265
f_micro w/ threshold uncertainty:     0.3020

f_macro:     0.1446
f_macro w/out threshold moving:     0.1293
f_macro w/ threshold uncertainty:     0.0665
---------------------------
####### TESTING UNFROZEN BEST_MODEL_MICRO #######
f_micro:     0.5572
f_micro w/out threshold moving:     0.5401
f_micro w/ threshold uncertainty:     0.2986

f_macro:     0.2389
f_macro w/out threshold moving:     0.1710
f_macro w/ threshold uncertainty:     0.0749
---------------------------
####### TESTING UNFROZEN BEST_MODEL_MICRO_NON_OPT #######
f_micro:     0.5580
f_micro w/out threshold moving:     0